# Injury in Australia/victoria due to drowning Cleaning Script

Data is has been imported from aihw.gov.au

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Loading the data into Data frames

In [2]:
sheet1 = pd.read_excel("aihw-Drowning-and-submersion-injury.xlsx", sheetname=1)
sheet1.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,"Table S1: Accidental drowning and submersion hospitalised injury cases, by age and sex, Australia, 2007–08 to 2016–17",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NaN,Age group (years),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0–4,5–9,10–14,15–19,20–24,25–29,30–34,35–39,40–44,...,55–59,60–64,65–69,70–74,75–79,80–84,85–89,90–94,95+,Total
2,Males,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007–08,131,22,15,19,19,24,12,15,17,...,6,7,3,4,3,2,0,0,1,317
4,2008–09,121,25,18,15,23,22,13,12,12,...,8,6,8,7,4,4,1,0,1,323


In [2]:
#Removing unnecessary columns at the end
sheet1 = sheet1[:35]

#Setting Genders value in a new column 
sheet1['Gender'] = [""]*sheet1.shape[0]

sheet1.iloc[2,-1] = sheet1.iloc[2,0][:-1]
sheet1.iloc[13,-1] = sheet1.iloc[13,0][:-1]
sheet1.iloc[24,-1] = sheet1.iloc[24,0][:-4]

#We have to fill all the values in the new column with respective genders
prev = ""
def setGender(string):
    global prev
    if(string == ""):
        return prev
    else:
        prev = string
        return string
sheet1['Gender'] = sheet1['Gender'].apply(setGender)

# Removing all na lines
droplist = []
for i in range(sheet1.shape[0]):
    if(pd.isna(sheet1.iloc[i,1])):
        droplist.append(i)

sheet1.drop(droplist,axis=0,inplace=True)
sheet1.iloc[1,0] = 'year'
sheet1.iloc[1,-1] = 'Gender'

#Changing the column names
colName = []
for i in range(0,sheet1.shape[1]):
    colName.append(sheet1.iloc[1,i])
sheet1.columns = colName
sheet1 = sheet1[2:]
#reseting the index
sheet1.reset_index(inplace = True)
sheet1.drop("index",axis=1,inplace=True)
sheet1.head()

,year,0–4,5–9,10–14,15–19,20–24,25–29,30–34,35–39,40–44,...,60–64,65–69,70–74,75–79,80–84,85–89,90–94,95+,Total,Gender
0,2007–08,131,22,15,19,19,24,12,15,17,...,7,3,4,3,2,0,0,1,317,Male
1,2008–09,121,25,18,15,23,22,13,12,12,...,6,8,7,4,4,1,0,1,323,Male
2,2009–10,130,21,13,25,32,24,15,14,10,...,10,9,6,11,2,0,1,1,358,Male
3,2010–11,119,15,14,19,20,15,14,13,18,...,11,3,5,4,3,2,1,0,311,Male
4,2011–12,107,21,22,15,28,10,17,14,14,...,4,4,8,9,3,1,1,0,301,Male


In [4]:
#removing all the unnecessary columns
#removing all the age groups above 20 as we will be using only data for agegroup below 20
dropCols = [ '20–24', '25–29', '30–34',
       '35–39', '40–44', '45–49', '50–54', '55–59', '60–64', '65–69', '70–74',
       '75–79', '80–84', '85–89', '90–94', '95+', 'Total']
sheet1.drop(dropCols,axis=1,inplace=True)
sheet1.head()

,year,0–4,5–9,10–14,15–19,Gender
0,2007–08,131,22,15,19,Male
1,2008–09,121,25,18,15,Male
2,2009–10,130,21,13,25,Male
3,2010–11,119,15,14,19,Male
4,2011–12,107,21,22,15,Male


In [5]:
# converting the year column from 2010-11 into 2011
def convertYear(string):
    tempyear = "20" + string[-2:]
    return int(tempyear)

sheet1['YEAR'] = sheet1['year'].apply(convertYear)
sheet1.drop('year',axis=1,inplace=True)
sheet1['year'] = sheet1['YEAR']
sheet1.drop('YEAR',axis=1,inplace=True)
sheet1 = sheet1.reindex(columns=['year','Gender','0–4', '5–9', '10–14', '15–19'])
sheet1 = sheet1[sheet1['Gender']!= 'Person']
sheet1.head()

,year,Gender,0–4,5–9,10–14,15–19
0,2008,Male,131,22,15,19
1,2009,Male,121,25,18,15
2,2010,Male,130,21,13,25
3,2011,Male,119,15,14,19
4,2012,Male,107,21,22,15


In [6]:
sheet1 = pd.melt(sheet1 ,id_vars=['year','Gender'],var_name='AgeGroup')
sheet1.rename(columns = {'value':'Injuries'}, inplace = True)
sheet1['Region'] = ['Australia']*(sheet1.shape[0])  
sheet1 = sheet1.reindex(columns=['year','Gender','Region','AgeGroup','Injuries'])
injuryAus = sheet1
injuryAus.head()

,year,Gender,Region,AgeGroup,Injuries
0,2008,Male,Australia,0–4,131
1,2009,Male,Australia,0–4,121
2,2010,Male,Australia,0–4,130
3,2011,Male,Australia,0–4,119
4,2012,Male,Australia,0–4,107


### Victoria

In [7]:
sheet1 = pd.read_excel("aihw-injcat-204-VIC_Supplementary-table.xlsx", sheetname=3)
sheet1.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,"Table S3: Hospitalised injury cases, by age, sex, and external cause, Victoria, 2016–17",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NaN,Age group (years),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0–4,5–9,10–14,15–19,20–24,25–29,30–34,35–39,40–44,...,55–59,60–64,65–69,70–74,75–79,80–84,85–89,90–94,95+,Total
2,Males,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Transport accidents,108,234,533,846,1116,1018,931,803,751,...,597,414,297,228,174,151,114,38,5,9769
4,Accidental drowning and submersion,9,6,3,7,11,3,5,3,5,...,2,3,0,0,0,0,1,0,0,62


In [8]:
#Removing unnecessary columns at the end
sheet1= sheet1[:44]

sheet1['Gender'] = [""]*sheet1.shape[0]

sheet1.iloc[2,-1] = sheet1.iloc[2,0][:-1]
sheet1.iloc[16,-1] = sheet1.iloc[16,0][:-1]
sheet1.iloc[30,-1] = sheet1.iloc[30,0][:-1]

#We have to fill all the values in the new column with respective genders
prev = ""
def setGender(string):
    global prev
    if(string == ""):
        return prev
    else:
        prev = string
        return string
sheet1['Gender'] = sheet1['Gender'].apply(setGender)

droplist = []
for i in range(sheet1.shape[0]):
    if(pd.isna(sheet1.iloc[i,1])):
        droplist.append(i)
droplist
sheet1.drop(droplist,axis=0,inplace=True)
sheet1.iloc[1,0] = 'Cause'
sheet1.iloc[1,-1] = 'Gender'

colName = []
for i in range(0,sheet1.shape[1]):
    colName.append(sheet1.iloc[1,i])
sheet1.columns = colName
sheet1 = sheet1[2:]

sheet1['Cause'] = sheet1['Cause'].str.lower()
sheet1 = sheet1[sheet1['Cause'].str.contains("drowning")]

sheet1.reset_index(inplace = True)
sheet1.drop("index",axis=1,inplace=True)
sheet1.head()

,Cause,0–4,5–9,10–14,15–19,20–24,25–29,30–34,35–39,40–44,...,60–64,65–69,70–74,75–79,80–84,85–89,90–94,95+,Total,Gender
0,accidental drowning and submersion,9,6,3,7,11,3,5,3,5,...,3,0,0,0,0,1,0,0,62,Male
1,accidental drowning and submersion,8,4,0,2,3,1,0,1,1,...,0,0,0,1,0,0,0,0,25,Female
2,accidental drowning and submersion,17,10,3,9,14,4,5,4,6,...,3,0,0,1,0,1,0,0,87,Person


In [9]:
sheet1.columns

Index(['Cause', '0–4', '5–9', '10–14', '15–19', '20–24', '25–29', '30–34',
       '35–39', '40–44', '45–49', '50–54', '55–59', '60–64', '65–69', '70–74',
       '75–79', '80–84', '85–89', '90–94', '95+', 'Total', 'Gender'],
      dtype='object')

In [10]:
dropCols = ['Cause', '20–24', '25–29', '30–34',
       '35–39', '40–44', '45–49', '50–54', '55–59', '60–64', '65–69', '70–74',
       '75–79', '80–84', '85–89', '90–94', '95+', 'Total']
sheet1.drop(dropCols,axis=1,inplace=True)
sheet1.head()

,0–4,5–9,10–14,15–19,Gender
0,9,6,3,7,Male
1,8,4,0,2,Female
2,17,10,3,9,Person


In [11]:
sheet1['year'] = [2017]*(sheet1.shape[0])
sheet1.head()

,0–4,5–9,10–14,15–19,Gender,year
0,9,6,3,7,Male,2017
1,8,4,0,2,Female,2017
2,17,10,3,9,Person,2017


In [12]:
sheet1 = pd.melt(sheet1 ,id_vars=['year','Gender'],var_name='AgeGroup')
sheet1.rename(columns = {'value':'Injuries'}, inplace = True)
sheet1 = sheet1.reindex(columns=['year','Gender','Region','AgeGroup','Injuries'])
sheet1['Region'] = ["Victoria"]*(sheet1.shape[0])
sheet1 = sheet1[sheet1['Gender']!= 'Person']
injuryVic = sheet1
injuryVic.head()

,year,Gender,Region,AgeGroup,Injuries
0,2017,Male,Victoria,0–4,9
1,2017,Female,Victoria,0–4,8
3,2017,Male,Victoria,5–9,6
4,2017,Female,Victoria,5–9,4
6,2017,Male,Victoria,10–14,3


In [13]:
injuryAus.head()

,year,Gender,Region,AgeGroup,Injuries
0,2008,Male,Australia,0–4,131
1,2009,Male,Australia,0–4,121
2,2010,Male,Australia,0–4,130
3,2011,Male,Australia,0–4,119
4,2012,Male,Australia,0–4,107


In [14]:
injury = injuryAus.append(injuryVic)
injury['Region'].unique()

array(['Australia', 'Victoria'], dtype=object)

In [15]:
injury['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [16]:
tempInjury = injury.set_index(['year', 'Gender','AgeGroup','Region'])

In [17]:
tempInjury.to_csv(r'db-injury-gender-20.csv')

### Location

In [18]:
injury.head()

,year,Gender,Region,AgeGroup,Injuries
0,2008,Male,Australia,0–4,131
1,2009,Male,Australia,0–4,121
2,2010,Male,Australia,0–4,130
3,2011,Male,Australia,0–4,119
4,2012,Male,Australia,0–4,107


In [19]:
sheet1 = pd.read_excel("aihw-Drowning-and-submersion-injury.xlsx", sheetname=4)
sheet1.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,"Table S4: Number of Accidental drowning and submersion hospitalised injury cases, by remoteness of usual residence, and age, Australia, 2012–13 to 2016–17",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,Age group (years),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0–4,5–9,10–14,15–19,20–24,25–29,30–34,35–39,40–44,45–49,50–54,55–59,60–64,65–69,70–74,75–79,80–84,85+,Total
2,Major cities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012–13,111,37,16,20,17,21,20,19,12,11,13,3,8,7,2,6,2,0,325
4,2013–14,131,29,24,16,23,23,14,12,10,10,10,10,5,8,3,11,7,1,347


In [20]:
sheet1['Location'] = [""]*sheet1.shape[0]

sheet1.iloc[2,-1] = sheet1.iloc[2,0]
sheet1.iloc[8,-1] = sheet1.iloc[8,0]
sheet1.iloc[14,-1] = sheet1.iloc[14,0]
sheet1.iloc[20,-1] = sheet1.iloc[20,0]
sheet1.iloc[26,-1] = sheet1.iloc[26,0]

prev = ""
def setLocation(string):
    global prev
    if(string == ""):
        return prev
    else:
        prev = string
        return string
sheet1['Location'] = sheet1['Location'].apply(setLocation)

droplist = []
for i in range(sheet1.shape[0]):
    if(pd.isna(sheet1.iloc[i,1])):
        droplist.append(i)
droplist
sheet1.drop(droplist,axis=0,inplace=True)
sheet1.iloc[1,0] = 'year'
sheet1.iloc[1,-1] = 'Location'

colName = []
for i in range(0,sheet1.shape[1]):
    colName.append(sheet1.iloc[1,i])
sheet1.columns = colName
sheet1 = sheet1[2:]



sheet1.reset_index(inplace = True)
sheet1.drop("index",axis=1,inplace=True)
sheet1.head()

,year,0–4,5–9,10–14,15–19,20–24,25–29,30–34,35–39,40–44,...,50–54,55–59,60–64,65–69,70–74,75–79,80–84,85+,Total,Location
0,2012–13,111,37,16,20,17,21,20,19,12,...,13,3,8,7,2,6,2,0,325,Major cities
1,2013–14,131,29,24,16,23,23,14,12,10,...,10,10,5,8,3,11,7,1,347,Major cities
2,2014–15,145,39,14,32,18,26,14,11,16,...,12,11,11,9,4,6,3,1,381,Major cities
3,2015–16,141,32,23,24,37,19,16,18,20,...,14,8,6,6,6,6,6,4,402,Major cities
4,2016–17,150,34,21,22,35,8,16,24,9,...,8,4,9,10,5,4,1,2,382,Major cities


In [21]:
sheet1.columns

Index(['year', '0–4', '5–9', '10–14', '15–19', '20–24', '25–29', '30–34',
       '35–39', '40–44', '45–49', '50–54', '55–59', '60–64', '65–69', '70–74',
       '75–79', '80–84', '85+', 'Total', 'Location'],
      dtype='object')

In [22]:
dropCols = [  '20–24', '25–29', '30–34',
       '35–39', '40–44', '45–49', '50–54', '55–59', '60–64', '65–69', '70–74',
       '75–79', '80–84', '85+', 'Total',]
sheet1.drop(dropCols,axis=1,inplace=True)
sheet1.head()

,year,0–4,5–9,10–14,15–19,Location
0,2012–13,111,37,16,20,Major cities
1,2013–14,131,29,24,16,Major cities
2,2014–15,145,39,14,32,Major cities
3,2015–16,141,32,23,24,Major cities
4,2016–17,150,34,21,22,Major cities


In [23]:
def convertYear(string):
    tempyear = "20" + string[-2:]
    return int(tempyear)

sheet1['YEAR'] = sheet1['year'].apply(convertYear)
sheet1.drop('year',axis=1,inplace=True)
sheet1['year'] = sheet1['YEAR']
sheet1.drop('YEAR',axis=1,inplace=True)
sheet1 = sheet1.reindex(columns=['year','Location','0–4', '5–9', '10–14', '15–19'])
sheet1.head()

,year,Location,0–4,5–9,10–14,15–19
0,2013,Major cities,111,37,16,20
1,2014,Major cities,131,29,24,16
2,2015,Major cities,145,39,14,32
3,2016,Major cities,141,32,23,24
4,2017,Major cities,150,34,21,22


In [24]:
sheet1 = pd.melt(sheet1 ,id_vars=['year','Location'],var_name='AgeGroup')
sheet1.rename(columns = {'value':'Injuries'}, inplace = True)

In [25]:
sheet1['Region'] = ['Australia']*(sheet1.shape[0])  
sheet1.head()

,year,Location,AgeGroup,Injuries,Region
0,2013,Major cities,0–4,111,Australia
1,2014,Major cities,0–4,131,Australia
2,2015,Major cities,0–4,145,Australia
3,2016,Major cities,0–4,141,Australia
4,2017,Major cities,0–4,150,Australia


In [26]:
sheet1 = sheet1.reindex(columns=['year','Location','Region','AgeGroup','Injuries'])

In [27]:
tempLocInjury = sheet1.set_index(['year','Location','Region','AgeGroup'])
tempLocInjury.to_csv(r'db-injury-loc-20.csv')

In [32]:
tempLocInjury.to_csv(r'db-injury-loc-20.csv')